<a href="https://colab.research.google.com/github/sqbitegh/Colabs/blob/main/peft_fine_tune_whisper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Peft Fine-Tune Whisper For Multilingual ASR with 🤗 Transformers

In this Colab, we present a step-by-step guide on how to fine-tune Whisper
for any multilingual ASR dataset using Hugging Face 🤗 Transformers. This is a
more "hands-on" version of the accompanying [blog post](https://huggingface.co/blog/fine-tune-whisper).
For a more in-depth explanation of Whisper, the Common Voice dataset and the theory behind fine-tuning, the reader is advised to refer to the blog post.

## Prepare Environment

We can verify that we've been assigned a GPU and view its specifications:

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

/bin/bash: line 1: nvidia-smi: command not found


We'll employ several popular Python packages to fine-tune the Whisper model.
We'll use `datasets[audio]` to download and prepare our training data, alongside
`transformers` and `accelerate` to load and train our Whisper model.
We'll also require the `soundfile` package to pre-process audio files,
`evaluate` and `jiwer` to assess the performance of our model, and
`tensorboard` to log our metrics. Finally, we'll use `gradio` to build a
flashy demo of our fine-tuned model.

In [ ]:
!pip install --upgrade --quiet pip
!pip install --upgrade --quiet datasets[audio] transformers accelerate evaluate jiwer tensorboard gradio

In [ ]:
!pip install huggingface_hub
#!pip install --upgrade --force-reinstall peft
!pip install bitsandbytes accelerate transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 61.1 MB/s eta 0:00:00


We strongly advise you to upload model checkpoints directly the [Hugging Face Hub](https://huggingface.co/)
whilst training. The Hub provides:
- Integrated version control: you can be sure that no model checkpoint is lost during training.
- Tensorboard logs: track important metrics over the course of training.
- Model cards: document what a model does and its intended use cases.
- Community: an easy way to share and collaborate with the community!

Linking the notebook to the Hub is straightforward - it simply requires entering your
Hub authentication token when prompted. Find your Hub authentication token [here](https://huggingface.co/settings/tokens):

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Error importing huggingface_hub._login: cannot import name 'is_colab_enterprise' from 'huggingface_hub.utils._runtime' (/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_runtime.py)


ImportError: cannot import name 'is_colab_enterprise' from 'huggingface_hub.utils._runtime' (/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_runtime.py)

## Load Dataset

Using 🤗 Datasets, downloading and preparing data is extremely simple.
We can download and prepare the Common Voice splits in just one line of code.

First, ensure you have accepted the terms of use on the Hugging Face Hub: [mozilla-foundation/common_voice_11_0](https://huggingface.co/datasets/mozilla-foundation/common_voice_11_0). Once you have accepted the terms, you will have full access to the dataset and be able to download the data locally.

Since Hindi is very low-resource, we'll combine the `train` and `validation`
splits to give approximately 8 hours of training data. We'll use the 4 hours
of `test` data as our held-out test set:

In [ ]:
from datasets import load_dataset, DatasetDict

common_voice_source = DatasetDict()
common_voice = DatasetDict()

#debug
common_voice_source["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "pl", split="train[:2%]+validation[:2%]")
common_voice_source["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "pl", split="test[:2%]")
#common_voice["validation"] = load_dataset("mozilla-foundation/common_voice_11_0", "pl", split="validation[:2%]")

#full
#common_voice_source["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "pl",split="train+validation")
#common_voice_source["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "pl", split="test")

print(common_voice_source)

DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 497
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 166
    })
})


Most ASR datasets only provide input audio samples (`audio`) and the
corresponding transcribed text (`sentence`). Common Voice contains additional
metadata information, such as `accent` and `locale`, which we can disregard for ASR.
Keeping the notebook as general as possible, we only consider the input audio and
transcribed text for fine-tuning, discarding the additional metadata information:

In [ ]:
common_voice_source = common_voice_source.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])

print(common_voice_source)

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 497
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 166
    })
})


## Prepare Feature Extractor, Tokenizer and Data

### Load WhisperFeatureExtractor

The Whisper feature extractor performs two operations:
1. Pads / truncates the audio inputs to 30s: any audio inputs shorter than 30s are padded to 30s with silence (zeros), and those longer that 30s are truncated to 30s
2. Converts the audio inputs to _log-Mel spectrogram_ input features, a visual representation of the audio and the form of the input expected by the Whisper model

We'll load the feature extractor from the pre-trained checkpoint with the default values:

In [ ]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-tiny")

In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-tiny", language="Polish", task="transcribe")

### Prepare Data

Let's print the first example of the Common Voice dataset to see
what form the data is in:

In [ ]:
print(common_voice_source["train"][0])

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/c35bb1e1b0a4cbcb6a811735439fdd61936c6f426fc9c1e20a2f0c00a00daf08/pl_train_0/common_voice_pl_20990285.mp3', 'array': array([ 0.00000000e+00,  3.58114180e-15, -4.51483935e-15, ...,
        2.94640543e-08, -1.09114431e-07, -3.94480764e-07]), 'sampling_rate': 48000}, 'sentence': 'Nie wiem właściwie, dlaczego tego nie zrobili.'}


Since
our input audio is sampled at 48kHz, we need to _downsample_ it to
16kHz prior to passing it to the Whisper feature extractor, 16kHz being the sampling rate expected by the Whisper model.

We'll set the audio inputs to the correct sampling rate using dataset's
[`cast_column`](https://huggingface.co/docs/datasets/package_reference/main_classes.html?highlight=cast_column#datasets.DatasetDict.cast_column)
method. This operation does not change the audio in-place,
but rather signals to `datasets` to resample audio samples _on the fly_ the
first time that they are loaded:

In [ ]:
from datasets import Audio

common_voice_source = common_voice_source.cast_column("audio", Audio(sampling_rate=16000))

Re-loading the first audio sample in the Common Voice dataset will resample
it to the desired sampling rate:

In [ ]:
print(common_voice_source["train"][0])

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/c35bb1e1b0a4cbcb6a811735439fdd61936c6f426fc9c1e20a2f0c00a00daf08/pl_train_0/common_voice_pl_20990285.mp3', 'array': array([ 1.81898940e-12, -2.27373675e-12, -2.72848411e-12, ...,
       -5.09688107e-07, -3.80561687e-07, -1.59896445e-07]), 'sampling_rate': 16000}, 'sentence': 'Nie wiem właściwie, dlaczego tego nie zrobili.'}


Now we can write a function to prepare our data ready for the model:
1. We load and resample the audio data by calling `batch["audio"]`. As explained above, 🤗 Datasets performs any necessary resampling operations on the fly.
2. We use the feature extractor to compute the log-Mel spectrogram input features from our 1-dimensional audio array.
3. We encode the transcriptions to label ids through the use of the tokenizer.

In [ ]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids

    return batch

We can apply the data preparation function to all of our training examples using dataset's `.map` method. The argument `num_proc` specifies how many CPU cores to use. Setting `num_proc` > 1 will enable multiprocessing. If the `.map` method hangs with multiprocessing, set `num_proc=1` and process the dataset sequentially.

In [ ]:
print(common_voice_source)
common_voice =  common_voice_source.map(prepare_dataset, remove_columns=common_voice_source.column_names["train"], num_proc=1)
print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 497
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 166
    })
})
DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 497
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 166
    })
})


In [ ]:
common_voice["train"]

Dataset({
    features: ['input_features', 'labels'],
    num_rows: 497
})

### Define a Data Collator

The data collator for a sequence-to-sequence speech model is unique in the sense that it
treats the `input_features` and `labels` independently: the  `input_features` must be
handled by the feature extractor and the `labels` by the tokenizer.

The `input_features` are already padded to 30s and converted to a log-Mel spectrogram
of fixed dimension by action of the feature extractor, so all we have to do is convert the `input_features`
to batched PyTorch tensors. We do this using the feature extractor's `.pad` method with `return_tensors=pt`.

The `labels` on the other hand are un-padded. We first pad the sequences
to the maximum length in the batch using the tokenizer's `.pad` method. The padding tokens
are then replaced by `-100` so that these tokens are **not** taken into account when
computing the loss. We then cut the BOS token from the start of the label sequence as we
append it later during training.

We can leverage the `WhisperProcessor` we defined earlier to perform both the
feature extractor and the tokenizer operations:

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch


Let's initialise the data collator we've just defined:

### Evaluation Metrics

We'll use the word error rate (WER) metric, the 'de-facto' metric for assessing
ASR systems. For more information, refer to the WER [docs](https://huggingface.co/metrics/wer). We'll load the WER metric from 🤗 Evaluate:

In [ ]:
import evaluate

metric = evaluate.load("wer")

We then simply have to define a function that takes our model
predictions and returns the WER metric. This function, called
`compute_metrics`, first replaces `-100` with the `pad_token_id`
in the `label_ids` (undoing the step we applied in the
data collator to ignore padded tokens correctly in the loss).
It then decodes the predicted and label ids to strings. Finally,
it computes the WER between the predictions and reference labels:

In [ ]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

### **Lora Specific Training**

In [ ]:
!pip install -q git+https://github.com/huggingface/peft.git@main

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, WhisperForConditionalGeneration
import torch
from peft import LoraConfig, get_peft_model



In [ ]:
# Assuming you're working with the Whisper model in your code
model_name = "openai/whisper-tiny"
#model = WhisperForConditionalGeneration.from_pretrained(model_name, load_in_8bit=False, device_map="auto")

#doesnt work
#from peft import prepare_model_for_int8_training
#model = prepare_model_for_int8_training(model, output_embedding_layer_name="proj_out")
#model = AutoModelForSeq2SeqLM.from_pretrained(model_name, load_in_8bit=True, device_map="auto")

firstrun = True
try:
    model = WhisperForConditionalGeneration.from_pretrained("ambind/whisper-tiny-pl", load_in_8bit=False, device_map="auto")
    firstrun = False

    #processor = WhisperProcessor.from_pretrained("ambind/whisper-tiny-pl") //processor recreated normally each time for now TODO: reinstate from saved model as below
    #config = AutoConfig.from_pretrained("ambind/whisper-tiny-pl")
    #model = WhisperForConditionalGeneration.from_pretrained("ambind/whisper-tiny-pl", config=config)
    print("Model loaded successfully!")

except (OSError, ValueError, EnvironmentError) as e:

    print("Please check your model ID and internet connection.")
    model = WhisperForConditionalGeneration.from_pretrained(model_name, load_in_8bit=False, device_map="auto")
    firstrun = True



tokenizer = AutoTokenizer.from_pretrained(model_name)


In [ ]:
def make_inputs_require_grad(module, input, output):
    output.requires_grad_(True)

model.model.encoder.conv1.register_forward_hook(make_inputs_require_grad)

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

In [ ]:
lora_config = LoraConfig(
     r=8,  # Rank of the LoRA update matrices
     lora_alpha=32,  # Scaling factor for the LoRA update
     lora_dropout=0.05,  # Dropout probability for the LoRA update
     bias="none",  # Whether to apply LoRA to the biases (none, all, or module_name)
     #task_type="SEQ_2_SEQ_LM",  # The task type (SEQ_2_SEQ_LM for Whisper)
     target_modules=["q_proj", "v_proj"], # This is a common choice for Whisper models

 )

In [ ]:
model = get_peft_model(model, lora_config)

now use model as above for training

In [ ]:
from transformers import Seq2SeqTrainingArguments


training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-tiny-lora-pl",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=2e-4, #debug
    warmup_steps = 50 if firstrun else 2, #debug  warmup_steps=50,
    max_steps=200, #debug max_steps=400,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    save_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=100, #save_steps=1000, #debug
    eval_steps=100, #eval_steps=1000, #debug
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
    remove_unused_columns=False,  # required as the PeftModel forward doesn't have the signature of the wrapped model's forward
    label_names=["labels"], #required for peft qlora to work
)

from transformers import Seq2SeqTrainer
from datasets import Dataset

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"], # eval_dataset=Dataset.from_dict(common_voice["test"][:100]), #debug
    data_collator=data_collator,
    #compute_metrics=compute_metrics,
    tokenizer= processor.feature_extractor,
)




<ipython-input-210-3afd2b4ea7a0>:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
max_steps is given, it will override any value given in num_train_epochs


In [ ]:
print(common_voice["train"])
#print(common_voice["train"][0])

trainer.train() # trainer.train(resume_from_checkpoint = True)

Dataset({
    features: ['input_features', 'labels'],
    num_rows: 497
})


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

# **=======================================================**

# **Stop here for lora , go down for vanila**  

```javascript
function ConnectButton(){
    console.log("Connect pushed");
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()
}
setInterval(ConnectButton, 60000);
```

In [ ]:
kwargs = {
    "dataset_tags": "mozilla-foundation/common_voice_11_0",
    "dataset": "Common Voice 11.0",  # a 'pretty' name for the training dataset
    "dataset_args": "config: pl, split: test",
    "language": "pl",
    "model_name": "Whisper tiny pl",  # a 'pretty' name for our model
    "finetuned_from": "openai/whisper-tiny",
    "tasks": "automatic-speech-recognition",
}

The training results can now be uploaded to the Hub. To do so, execute the `push_to_hub` command and save the preprocessor object we created:

In [ ]:
trainer.push_to_hub(**kwargs)

NameError: name 'trainer' is not defined

## Building a Demo

Now that we've fine-tuned our model we can build a demo to show
off its ASR capabilities! We'll make use of 🤗 Transformers
`pipeline`, which will take care of the entire ASR pipeline,
right from pre-processing the audio inputs to decoding the
model predictions.

Running the example below will generate a Gradio demo where we
can record speech through the microphone of our computer and input it to
our fine-tuned Whisper model to transcribe the corresponding text:

In [ ]:
from transformers import pipeline
import gradio as gr

pipe = pipeline(model="sanchit-gandhi/whisper-small-hi")  # change to "your-username/the-name-you-picked"

def transcribe(audio):
    text = pipe(audio)["text"]
    return text

iface = gr.Interface(
    fn=transcribe,
    inputs=gr.Audio(source="microphone", type="filepath"),
    outputs="text",
    title="Whisper Small Hindi",
    description="Realtime demo for Hindi speech recognition using a fine-tuned Whisper small model.",
)

iface.launch()